In [ ]:
import sys
sys.path.append('../input/wav2vec-asr-utils')

In [ ]:
!pip install -U datasets -q
!pip install fsspec==2021.4.0 -q

In [ ]:
import pandas as pd
from datasets import load_dataset, load_metric,Dataset
from tqdm import tqdm
import torch
import soundfile as sf
import torchaudio
from transformers import Wav2Vec2ForCTC
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2CTCTokenizer

import warnings
warnings.filterwarnings('ignore')

In [ ]:
val =pd.read_csv("../input/automatic-speech-recognition-in-wolof/Test.csv")
val["path"] = "../input/automatic-speech-recognition-in-wolof/Noise Removed/tmp/WOLOF_ASR_dataset/noise_remove/"+val["ID"]+".wav"
val.rename(columns = {'transcription':'sentence'}, inplace = True)
common_voice_val = Dataset.from_pandas(val)

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("../input/wav2vec2largexlsrwolof-model/Full-Base-wav2vec2-large-xlsr-WOLOF/wav2vec2-large-xlsr-WOLOF").to("cuda")

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("../input/wav2vec2largexlsrwolof-model/Full-Base-wav2vec2-large-xlsr-WOLOF/wav2vec2-large-xlsr-WOLOF")

In [ ]:
# common_voice_val = Dataset.load_from_disk("../input/automatic-speech-recognition-in-wolof/perproc_data/Submission")

In [ ]:
def speech_file_to_array_fn_test(batch):
    speech_array, sampling_rate = sf.read(batch["path"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
#     batch["target_text"] = batch["sentence"]
    return batch

In [ ]:
def prepare_dataset_test(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], padding=True,sampling_rate=batch["sampling_rate"][0]).input_values
    
#     with processor.as_target_processor():
#         batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
common_voice_val = common_voice_val.remove_columns([ "ID","age",  "down_votes", "gender",  "up_votes"])

In [ ]:
common_voice_val = common_voice_val.map(speech_file_to_array_fn_test, remove_columns=common_voice_val.column_names)

In [ ]:
common_voice_val = common_voice_val.map(prepare_dataset_test, remove_columns=common_voice_val.column_names, batch_size=8, num_proc=4, batched=True)

In [ ]:
final_pred = []
for i in tqdm(range(common_voice_val.shape[0])):    
    input_dict = processor(common_voice_val[i]["input_values"], return_tensors="pt", padding=True)

    logits = model(input_dict.input_values.to("cuda")).logits

    pred_ids = torch.argmax(logits, dim=-1)[0]
    prediction = processor.decode(pred_ids)
    final_pred.append(prediction)

In [ ]:
val["transcription"] = final_pred
val["transcription"] = val["transcription"].str.capitalize()
val.iloc[1390,6] = "Mosquée de"


val[["ID","transcription"]].to_csv("submission16.csv",index=False)
val.head()